In [ ]:
import yt_dlp
import os
from pathlib import Path
import subprocess


def check_ffmpeg():
    """检查FFmpeg是否安装"""
    try:
        result = subprocess.run(['ffmpeg', '-version'],
                                capture_output=True,
                                text=True,
                                creationflags=subprocess.CREATE_NO_WINDOW)
        return "ffmpeg version" in result.stdout
    except:
        return False


# 配置FFmpeg路径（自动检测）
FFMPEG_PATH = 'ffmpeg'
if os.path.exists('C:/ffmpeg/bin/ffmpeg.exe'):
    FFMPEG_PATH = 'C:/ffmpeg/bin/ffmpeg.exe'


# 自定义文件名处理器
def clean_title(title):
    """只保留安全字符，但不替换空格"""
    keep_chars = (' ', '.', '_', '-')
    return "".join(c for c in title if c.isalnum() or c in keep_chars).strip()


# 下载配置
ydl_opts = {
    # 最佳MP4视频+最佳音频
    'format': '(bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best)',
    'merge_output_format': 'mp4',

    # 文件保存路径 - 每个视频单独文件夹
    'outtmpl': {
        'default': os.path.join('D:/Downloads/yt-dl_Downloads', '%(title)s', '%(title)s.%(ext)s'),
        'thumbnail': os.path.join('D:/Downloads/yt-dl_Downloads', '%(title)s', '%(title)s.%(ext)s'),
        # 'subtitle': os.path.join('D:/Downloads/yt-dl_Downloads', '%(title)s', '%(title)s.%(lang)s.%(ext)s'),
    },

    # 修改默认文件名处理行为
    'restrictfilenames': False,  # 禁用严格文件名限制
    'windowsfilenames': False,  # 不禁用Windows保留字符

    # 后处理函数
    'postprocessor_hooks': [
        lambda info: {
            'title': clean_title(info.get('title', ''))
        }
    ],

    # 缩略图设置
    'writethumbnail': True,
    'embedthumbnail': False,

    # 字幕设置
    # 'writesubtitles': True,
    # 'writeautomaticsub': True,
    # 'subtitleslangs': ['zh', 'en', 'zh-Hans', 'zh-Hant'],
    # 'subtitlesformat': 'srt',
    # 'embedsubtitles': False,

    # 后处理器
    'postprocessors': [
        {
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4'
        },
        {
            'key': 'FFmpegMetadata'
        }
    ],

    # 系统配置
    'ffmpeg_location': FFMPEG_PATH,
    'ignoreerrors': True,
    'nooverwrites': True,
    'quiet': False,
}


def ensure_dirs(base_path):
    """确保基础目录存在"""
    Path(base_path).mkdir(parents=True, exist_ok=True)


def read_urls(file_path):
    """从文件读取URL列表"""
    if not os.path.exists(file_path):
        print(f"错误：文件 {file_path} 不存在")
        return []

    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip() and not line.startswith('#')]


def main():
    print("YouTube视频下载器（纯净版）")
    print("=" * 40)

    # 检查FFmpeg
    if not check_ffmpeg():
        print("错误：FFmpeg未正确安装")
        print("请从 https://ffmpeg.org 下载并添加到PATH")
        print("或解压到 C:\\ffmpeg\\")
        return

    # 准备基础目录
    base_dir = 'D:/Downloads/yt-dl_Downloads'
    ensure_dirs(base_dir)

    # 读取URL
    urls = read_urls('CS50x2025.txt')
    if not urls:
        print("错误：urls.txt 中没有有效的URL")
        print("请每行添加一个YouTube链接，空行和#开头的行会被忽略")
        return

    print(f"准备下载 {len(urls)} 个视频...")

    # 开始下载
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(urls)
    except Exception as e:
        print(f"下载出错: {str(e)}")

    print("\n下载完成！")
    print(f"文件保存在: {base_dir}")
    print("目录结构示例:")
    print("  视频标题/")
    print("    ├── 视频标题.mp4")
    print("    ├── 视频标题.webp")
    print("    ├── 视频标题.zh.srt")
    print("    └── 视频标题.en.srt")


if __name__ == '__main__':
    main()